# Correlation Analysis Workflow
This notebook contains the workflow for validating historical time series data from observed data points and modelled historical streamflow (MHS) data.  The MHS data can be accessed through the Streamflow Prediction Tool (SPT) on the Tethys platfrom. To successfully run the workflow, the following modules must be installed.

In [0]:
#import necessary modules
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from tqdm import tqdm
import glob
import os
import matplotlib.pyplot as plt
import hydrostats as hs

## Functions
The following are the functions defined within the workflow to help organize and run the different metrics

### plot_station()
The plot_station() generates a plot for each station and saves it as a .png file within the Plots folder, which is also created as part of the function. 

In [0]:
def plot_station(dataframe, stations, folderpath):
    # Create folder for plot files to live
    plt_filename = folderpath + "//Plots//" + str(stations) + "_plot.png"
    if not os.path.exists(folderpath + "//Plots"):
        os.makedirs(folderpath + "//Plots//")
    comp_plot = dataframe[['Observed Flow', 'Predicted Flow']].plot(legend=True, color=['blue', 'green'])
    comp_plot.set_title('Observed and Predicted Flow for ' + stations)
    comp_plot.set_ylabel('Flow (cms)')
    plt.savefig(plt_filename)

### correlation_stats()
The correlation_stats function takes a dataframe (df) of merged data, an adjustment which accounts for any negatives, and a beginning and ending timestamp, then calculates all of the metrics on the dataframe. 
This function calculates the metrics on both a monthly and yearly basis.  The monthly statistic is the result of all the datapoints within a specific month, while the yearly statistic is the result of the entire dataset. The correlation_stats() function calculates the correlation coefficient, mean difference, mean variance, as well as the average observed and predicted flows. 

In [0]:
def correlation_stats(df, adjustment, beg, end):
    # create Dataframe columns
    cor_coeff = {}
    mean_diff = {}
    mean_var = {}
    month_predicted = {}
    month_observed = {}
    # Monthly metrics
    for i in range(beg, end):
        log_month_predicted = df.where(df.date.dt.month == i).dropna()['log Predicted']
        log_month_observed = df.where(df.date.dt.month == i).dropna()['log Recorded']
        cor_coeff[i] = hs.acc(log_month_observed, log_month_predicted)
        mean_diff[i] = np.exp(scipy.stats.gmean(log_month_observed) - scipy.stats.gmean(log_month_predicted))
        mean_var[i] = np.var(log_month_observed - log_month_predicted)
        month_predicted[i] = np.exp(np.mean(log_month_predicted - adjustment))
        month_observed[i] = np.exp(np.mean(log_month_observed - adjustment))
    # Yearly Metrics
    log_pred = df.dropna()['log Predicted']
    log_observed = df.dropna()['log Recorded']
    cor_coeff[14] = hs.acc(log_observed, log_pred)
    mean_diff[14] = np.exp(scipy.stats.gmean(log_observed) - scipy.stats.gmean(log_pred))
    mean_var[14] = np.var(log_observed - log_pred)
    month_predicted[14] = np.exp(np.mean(log_pred - adjustment))
    month_observed[14] = np.exp(np.mean(log_observed - adjustment))

    # Define dataframe columns
    cor_coeff_df = pd.DataFrame.from_dict(cor_coeff, orient='Index')
    mean_df = pd.DataFrame.from_dict(mean_diff, orient='Index')
    variance_df = pd.DataFrame.from_dict(mean_var, orient='Index')
    predicted_df = pd.DataFrame.from_dict(month_predicted, orient='Index')
    observed_df = pd.DataFrame.from_dict(month_observed, orient='Index')
    correlation_df = pd.concat([cor_coeff_df, mean_df, observed_df, predicted_df, variance_df], axis=1)
    # print(correlation_df)
    correlation_df.columns = ['Correlation', 'Mean Difference', 'Observed Flow', 'Predicted Flow',
                              'Mean Variance']
    return correlation_df

### error_stats()
The error_stats is very similar to the correlation_stats() function, except it calculates the RMSE, RMSLE, NSE, R^2, and spectral angle coefficients for the dataframe. 

In [0]:
def error_stats(df, beg, end):
    rmse = {}
    rmse_log = {}
    NS_eff = {}
    R2 = {}
    sa = {}
    # calculate metrics by month
    for i in range(beg, end):
        predicted = df.where(df.date.dt.month == i).dropna()['predicted streamflow']
        observed = df.where(df.date.dt.month == i).dropna()['recorded streamflow']
        log_month_predicted = df.where(df.date.dt.month == i).dropna()['log Predicted']
        log_month_observed = df.where(df.date.dt.month == i).dropna()['log Recorded']
        n = predicted.count()
        # print n
        rmse[i] = hs.rmse(predicted, observed)
        rmse_log[i] = hs.rmsle(log_month_predicted, log_month_observed)
        NS_eff[i] = hs.E(log_month_predicted, log_month_observed)
        R2[i] = hs.r_squared(log_month_predicted, log_month_observed)
        sa[i] = hs.sa(predicted, observed)
    # Calculate metrics for year
    predicted = df.dropna()['predicted streamflow']
    observed = df.dropna()['recorded streamflow']
    log_predicted = df.dropna()['log Predicted']
    log_observed = df.dropna()['log Recorded']
    n = predicted.count()
    rmse[14] = hs.rmse(predicted, observed)
    rmse_log[14] = hs.rmsle(log_predicted, log_observed)
    NS_eff[14] = hs.E(log_predicted, log_observed)
    R2[14] = hs.r_squared(predicted, observed)
    sa[14] = hs.sa(predicted, observed)

    rmse_df = pd.DataFrame.from_dict(rmse, orient='Index')
    rmse_log_df = pd.DataFrame.from_dict(rmse_log, orient='Index')
    NS_eff_df = pd.DataFrame.from_dict(NS_eff, orient='Index')
    R2_df = pd.DataFrame.from_dict(R2, orient='Index')
    sa_df = pd.DataFrame.from_dict(sa, orient='Index')
    error_df = pd.concat([rmse_df, rmse_log_df, NS_eff_df, R2_df, sa_df], axis=1)

    error_df.columns = ['RMSE', 'Log RMSE', 'Nash-Sutcliffe Efficiency', 'R^2 Coefficient', 'Spectral Angle']
    return error_df

### monthly_stats_analysis()
The monthly_stats_analysis() function takes the list of stations, the folderpath to the data folder, and the datapath to the actual file, accounts for any negative or zero values within the merged dataset, and runs the correlation_stats() and error_stats() functions.  The resulting dataframe from these two functions are then merged to form a final dataframe, which is printed to a .csv file. 

In [0]:
def monthly_stats_analysis(stations, folderpath, datapath):
    df = pd.read_csv(datapath)
    df['recorded streamflow'] = df['recorded streamflow'].astype(np.float64)

    # Replace 0's with .001
    df = df.replace(to_replace=0, value=0.001)
    df = df.replace(to_replace=1, value=1.0001)

    # Log Transform Data
    df['log Predicted'] = np.log(df['predicted streamflow'])
    df['log Recorded'] = np.log(df['recorded streamflow'], dtype='float64')

    # Account for any negatives:
    if min(df['log Predicted']) < 0 or min(df['log Recorded']) < 0:
        adjustment = max(abs(min(df['log Predicted'])), abs(min(df['log Recorded']))) + 1
        print(adjustment)
        df['log Predicted'] = df['log Predicted'] + adjustment
        df['log Recorded'] = df['log Recorded'] + adjustment
    else:
        adjustment = 0

    # Sort by Months
    df['date'] = pd.to_datetime(df['Datetime'])
    df.drop('Datetime', axis=1, inplace=True)
    df['difference'] = df[['log Recorded']].sub(df['log Predicted'], axis=0)

    # Correlation Statistics
    # Months are from 1 to 12, 13 is to calculate yearly statistics
    correlation_df = correlation_stats(df, adjustment, 1, 13)
    # Error Metrics
    error_df = error_stats(df, 1, 13)

    # Combine into one dataframe
    results_df = pd.concat([correlation_df, error_df], axis=1)
    results_df.index.name = "Month"
    results_df['Station'] = stations
    print(results_df)

    # Plot Results
    plot_station(results_df, stations, folderpath)

    # Average values
    av_cor = results_df['Correlation'].mean()
    av_mean = results_df['Mean Difference'].mean()
    av_var = results_df['Mean Variance'].mean()
    pred_value = results_df['Predicted Flow']
    obs_value = results_df['Observed Flow']
    rmse_val = hs.rmse(pred_value, obs_value)
    print(av_cor, av_mean, av_var, rmse_val)

    # Print Results
    datapath_results = folderpath + "//Results//" + stations + "_results.csv"
    if not os.path.exists(folderpath + "//Results"):
        os.makedirs(folderpath + "//Results")
    results_df.to_csv(datapath_results, sep=',', index_label="Month")

### combine_csvs()
The combine_csvs() function takes the individual results file from each station and merges them into a single summary file.  This file is saved in the same folder as the individual results. 

In [0]:
def combine_csvs(folderpath):
    # Creates summary csv files for each folder of merged datafiles
    results = glob.glob(folderpath + "*_results.csv")
    df_list = []
    for file in tqdm(sorted(results)):
        df_list.append(pd.read_csv(file))
    df_results = pd.concat(df_list)
    df_results.to_csv(folderpath + 'National Results.csv', sep=',', index=False)

## Running the Workflow
### Set variable names
Set the variable foldername to be the name of the folder where the merged data files are found.  The folderpath sets the directory where the rest of the analysis will be run. 
The workflow then makes a list of the stations found, and loops through that list to run the metrics for each station. 

In [0]:
foldername="\\Peru\\"
folderpath="D:\Jackson\Streamflow Prediction\Data Analysis\Python Stats" + foldername
# Create list of stations in folder
stations = [os.path.basename(x) for x in glob.glob(folderpath + "*_merged.csv")]
stations = [s.replace('_merged.csv', '') for s in stations]

### Loop through stations
For each station in the stations list, the workflow runs the monthly_stats_analysis function to calculate the metrics for each station, create a plot of the monthly flow values, and also creates a final summary file.

In [0]:
for i in stations:
    station = str(i)
    print(station)
    datapath = str(folderpath) + station + "_merged.csv"
    monthly_stats_analysis(station, folderpath, datapath)

## Running the Workflow
To use the above functions together with the named variables and the loop function, select the code block above with the loop function, and use the command "ctrl+F8" or "Run Before" command.  This tells the notebook to run all of the code blocks above the cursor.  This initializes all the previously discussed functions, and allows the Python to complete the loop command. 

# Lag Analysis Workflow
The lag analysis workflow takes the MHS data, interpolates it using a cubic spline, and then lags it by consecutive time steps to calculate the maximum lag necessary for the optimum spectral angle and R^2 coefficients.  The number of time steps can be specified by the user. The results from the lag analysis are output as .csv files. 

The user must input:
1. file_location = directory where the merged data files are saved. This should be a specific folder of files.
2. result_location = directory where the results from the analysis will be saved.
3. num_timesteps = number of timesteps for the lag analysis to consider. A timestep is equal to 6 hours.

In [0]:
import numpy as np
import pandas as pd
from scipy import interpolate
import os
from os import listdir

#Define file locations
file_location="D:\Jackson\Streamflow Prediction\Data Analysis\Python Stats\Peru\Merged Data\\"
result_location='D:\Jackson\Streamflow Prediction\Data Analysis\Python Stats\Peru\Results\\'
num_timesteps = 15

def r_squared(x, y):
    return np.corrcoef(x, y)[0, 1] ** 2


# importing data as a data-frame
def r_squared_time_lag(merged_data_path, location, n=num_timesteps):
    # Importing Data to arrays
    predicted_streamflow = np.genfromtxt(merged_data_path, dtype='float', delimiter=",", skip_header=1, usecols=1)
    recorded_streamflow = np.genfromtxt(merged_data_path, dtype='float', delimiter=",", skip_header=1, usecols=2)
    # Creating new arrays for interpolation
    x1 = np.arange(predicted_streamflow.size)
    x2 = np.arange(recorded_streamflow.size)
    # Cubic spline interpolation for predicted streamflow
    tck1 = interpolate.splrep(x1, predicted_streamflow, s=0)
    xnew1 = np.arange(0, x1.size-.75, 1 / 4)
    predicted_interp = interpolate.splev(xnew1, tck1, der=0)
    # cubic spline interpolation for recorded streamflow
    tck2 = interpolate.splrep(x2, recorded_streamflow, s=0)
    xnew2 = np.arange(0, x2.size-0.75, 1 / 4)
    recorded_interp = interpolate.splev(xnew2, tck2, der=0)
    # creating an empty array for spectral values
    y = np.zeros((n + 1)*2 - 1)
    # Looping through the different times with roll function
    for lag in range(-n, n + 1):
        value = r_squared(np.roll(recorded_interp, lag), predicted_interp)
        y[lag + n] = value
    return y.max(), y.argmax() - n, location


# Creating a list for locations and file paths
merged_files = listdir(file_location)
print(merged_files)
locations = []
for i in range(len(merged_files)):
    name = merged_files[i][:merged_files[i].find('_')]
    locations.append(name)
for i in range(len(merged_files)):
    merged_files[i] = file_location + merged_files[i]


y_max_array = np.zeros(len(locations))
y_max_location_array = np.zeros(len(locations))

for i, j, k in zip(merged_files, locations, range(len(locations))):
    y_max_array[k], y_max_location_array[k], _ = r_squared_time_lag(i, j)

value_array = np.column_stack((y_max_array, y_max_location_array))

summary_df = pd.DataFrame(data=value_array, index=locations, columns=["Max R^2 Value", "Time Shift at Max R^2 Value"])

summary_df.to_csv(result_location+'\\r_squared_summary_NP.csv', index_label="Locations")


def spectral(x, y):
    value = np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
    return value


# importing data as a data-frame
def spectral_time_lag(merged_data_path, location, n=num_timesteps):
    # Importing Data to arrays
    predicted_streamflow = np.genfromtxt(merged_data_path, dtype='float', delimiter=",", skip_header=1, usecols=1)
    recorded_streamflow = np.genfromtxt(merged_data_path, dtype='float', delimiter=",", skip_header=1, usecols=2)
    # Creating new arrays for interpolation
    x1 = np.arange(predicted_streamflow.size)
    x2 = np.arange(recorded_streamflow.size)
    # Cubic spline interpolation for predicted streamflow
    tck1 = interpolate.splrep(x1, predicted_streamflow, s=0)
    xnew1 = np.arange(0, x1.size, 1 / 4)
    predicted_interp = interpolate.splev(xnew1, tck1, der=0)
    # cubic spline interpolation for recorded streamflow
    tck2 = interpolate.splrep(x2, recorded_streamflow, s=0)
    xnew2 = np.arange(0, x2.size, 1 / 4)
    recorded_interp = interpolate.splev(xnew2, tck2, der=0)
    # creating an empty array for spectral values
    y = np.zeros((n + 1)*2 - 1)
    for i in range(-n, n + 1):
        value = spectral(np.roll(recorded_interp, i), predicted_interp)
        y[i + n] = value
    return location, y.max(), y.argmax() - n


# Creating a list for locations and file paths
merged_files = listdir(file_location)
locations = []
for i in range(len(merged_files)):
    name = merged_files[i][:merged_files[i].find('_')]
    locations.append(name)
for i in range(len(merged_files)):
    merged_files[i] = file_location + merged_files[i]

y_max_array = np.zeros(len(locations))
y_max_location_array = np.zeros(len(locations))

for i, j, k in zip(merged_files, locations, range(len(locations))):
    _, y_max_array[k], y_max_location_array[k] = spectral_time_lag(i, j)

value_array = np.column_stack((y_max_array, y_max_location_array))

summary_df = pd.DataFrame(data=value_array, index=locations, columns=["Max Spectral Value", "Time Shift at Max "
                                                                                            "Spectral Value"])

summary_df.to_csv(result_location+'\\Spectral_Summary_NP.csv', index_label="Locations")